In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
accidents = pd.read_csv('../data/traffic_accidents.csv')

In [4]:
accidents.dtypes

offense_id                         int64
dist_from_signal                 float64
top_traffic_accident_offense      object
reported_date                     object
incident_address                  object
lon                              float64
lat                              float64
district_id                       object
precinct_id                      float64
neighborhood_id                   object
bicycle_ind                      float64
pedestrian_ind                   float64
HARMFUL_EVENT_SEQ_MOST            object
ROAD_DESCRIPTION                  object
ROAD_CONDITION                    object
LIGHT_CONDITION                   object
TU1_VEHICLE_TYPE                  object
TU1_TRAVEL_DIRECTION              object
TU1_VEHICLE_MOVEMENT              object
TU1_DRIVER_ACTION                 object
TU1_DRIVER_HUMANCONTRIBFACTOR     object
TU1_PEDESTRIAN_ACTION             object
TU2_VEHICLE_TYPE                  object
TU2_TRAVEL_DIRECTION              object
TU2_VEHICLE_MOVE

### Data from 1-1-2013 to 6-3-2024

In [5]:
accidents['top_traffic_accident_offense'].value_counts()

top_traffic_accident_offense
TRAF - ACCIDENT                   145445
TRAF - ACCIDENT - HIT & RUN        64684
TRAF - ACCIDENT - DUI/DUID          6057
TRAF - ACCIDENT - SBI               3494
TRAF - ACCIDENT - POLICE            2193
TRAF - ACCIDENT - FATAL              629
Name: count, dtype: int64

In [6]:
accidents['LIGHT_CONDITION'].value_counts()

LIGHT_CONDITION
DAY LIGHT              123343
DARK-LIGHTED            39946
Daylight                28474
Dark-Lighted            10347
DAWN OR DUSK             7394
                         5384
DARK-UNLIGHTED           3851
Dawn or Dusk             1422
Dark-Unlighted           1180
UNDER INVESTIGATION       832
Name: count, dtype: int64

In [7]:
accidents.head(2)

,offense_id,dist_from_signal,top_traffic_accident_offense,reported_date,incident_address,lon,lat,district_id,precinct_id,neighborhood_id,bicycle_ind,pedestrian_ind,HARMFUL_EVENT_SEQ_MOST,ROAD_DESCRIPTION,ROAD_CONDITION,LIGHT_CONDITION,TU1_VEHICLE_TYPE,TU1_TRAVEL_DIRECTION,TU1_VEHICLE_MOVEMENT,TU1_DRIVER_ACTION,TU1_DRIVER_HUMANCONTRIBFACTOR,TU1_PEDESTRIAN_ACTION,TU2_VEHICLE_TYPE,TU2_TRAVEL_DIRECTION,TU2_VEHICLE_MOVEMENT,TU2_DRIVER_ACTION,TU2_DRIVER_HUMANCONTRIBFACTOR,TU2_PEDESTRIAN_ACTION,SERIOUSLY_INJURED,FATALITIES,FATALITY_MODE_1,FATALITY_MODE_2,SERIOUSLY_INJURED_MODE_1,SERIOUSLY_INJURED_MODE_2,POINT_X,POINT_Y
0,202412503554010,0.0,TRAF - ACCIDENT - HIT & RUN,3/5/24 15:07,E SPEER BLVD / N GRANT ST,-104.983794,39.723423,3,311.0,Speer,0.0,0.0,Front to Rear,Intersection Related,Dry,Daylight,Passenger Car/Passenger Van,East,Going Straight,Careless Driving,Aggressive Driving,NaN,Passenger Car/Passenger Van,East,Going Straight,No Contributing Action,No Apparent Contributing Factor,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,202323009454010,0.0,TRAF - ACCIDENT - HIT & RUN,5/27/23 22:21,E SPEER BLVD / N GRANT ST,-104.983794,39.723423,3,311.0,Speer,0.0,0.0,Side to Side-Same Direction,At Intersection,Dry,Dark-Lighted,SUV,South,Other (Describe in Narrative),NaN,Not Observed,NaN,Passenger Car/Passenger Van,South,Going Straight,No Contributing Action,Not Observed,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### Converting to Datetime and extracting months and years

In [8]:
accidents['reported_date'] = pd.to_datetime(accidents['reported_date'], format='ISO8601')
accidents['month'] = accidents['reported_date'].dt.month
accidents['month_name'] = accidents['reported_date'].dt.month_name()
accidents['year'] = accidents['reported_date'].dt.year

ValueError: Time data 3/5/24 15:07 is not ISO8601 format, at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
accidents.shape

In [ ]:
accidents['reported_date'].describe

In [ ]:
accidents['month'] = accidents['reported_date'].dt.month
accidents['month_name'] = accidents['reported_date'].dt.month_name()
accidents['year'] = accidents['reported_date'].dt.year

In [ ]:
over_time = accidents.groupby('year').agg({'offense_id' : 'nunique'}).reset_index()

In [ ]:
over_time

In [ ]:
accidents['offense_id'].nunique()

In [ ]:
plt.plot(over_time['year'], over_time['offense_id'])

In [ ]:
accidents.to_csv('../data/accidents_clean.csv')